In [ ]:
import cv2
import numpy as np
import insightface
from matplotlib import pyplot as plt

# CODE: https://github.com/hukkelas/DSFD-Pytorch-Inference

# Process the face detection on a video
1. Load the numpy array generated from a video
1. Process the face detectio
1. Visual check of the performances

## Load the numpy array : 3D + 3D numpy with colors or 3D if gray

In [ ]:
USE_COLOR = True 
SHOW_DET_FACES = True
fps_np = 1.

In [ ]:
# Load the numpy array 
path_arrays = "../data/arrays/"
array_name = 'BA_GNS_' + str(int(fps_np)) + 'fps'
#array_name = 'EJ_GNS_' + str(int(fps_np)) + 'fps'
if USE_COLOR:
    array_name += '_RGB'
else:
    array_name += '_Gray'
file_out =  path_arrays + array_name + '.npy'

video_np = np.load(file = file_out)

video_np = video_np.astype(np.uint8)

In [ ]:
# check the size of the video 
if USE_COLOR:
    nb_frame, height, width, color = video_np.shape
else:
    nb_frame, height, width = video_np.shape
    color = 1
    
print('Number of frame : ' + str(nb_frame))
print('Frame size WxH : ' + str(width) + 'x' + str(height))
print('Colors : ' + str(color))

## Process the video

In [ ]:
# Hyperparamters 

# contour margin for the face cropping
margin = 0

# mininum size of the face to be 
#min_face_size = 20

In [ ]:
model_dir_path = insightface.utils.download("models", "buffalo_l")
detector = insightface.model_zoo.SCRFD(model_file=model_dir_path+"/det_10g.onnx")
detector.prepare(-1)

In [ ]:
# outputs
crop_faces = []
no_face = []
video_fd = np.copy(video_np)

In [ ]:
for idx, frame_idx in enumerate(video_np):
    video_fd[idx, :] = frame_idx
    bboxes, kpss = detector.detect(frame_idx, (640, 640))
    if len(bboxes) == 0:
        no_face.append(frame_idx)
    else:
        for frame_idx_bbox, frame_idx_kps in zip(bboxes, kpss):
            x0, y0, x1, y1 = [int(_) for _ in frame_idx_bbox[:4]]
            
            x = int(max(x0 - margin/2, 0))
            y = int(max(y0 - margin/2, 0))
            h = y1 - y0 + margin
            w = x1 - x0 + margin
            
            # save the crop images
            crop_face = frame_idx[y:y+h, x:x+w]
            crop_faces.append(crop_face)
            
            # put a block box on the faces to make the detection more visible
            cv2.rectangle(video_fd[idx], (x0, y0), (x1, y1), (0, 255, 0), 2)
            (left_eye_x, left_eye_y) = int(frame_idx_kps[0][0]), int(frame_idx_kps[0][1]) # left eye center
            (right_eye_x, right_eye_y) = int(frame_idx_kps[1][0]), int(frame_idx_kps[1][1]) # right eye center
            (nose_tip_x, nose_tip_y) = int(frame_idx_kps[2][0]), int(frame_idx_kps[2][1]) # nose tip
            (mouth_left_x, mouth_left_y) = int(frame_idx_kps[3][0]), int(frame_idx_kps[3][1]) # mouth left corner
            (mouth_right_x, mouth_right_y) = int(frame_idx_kps[4][0]), int(frame_idx_kps[4][1]) # mouth right corner
            cv2.circle(video_fd[idx], (left_eye_x, left_eye_y), 1, (0, 0, 255), 4) 
            cv2.circle(video_fd[idx], (right_eye_x, right_eye_y), 1, (0, 255, 255), 4) 
            cv2.circle(video_fd[idx], (nose_tip_x, nose_tip_y), 1, (255, 0, 255), 4) 
            cv2.circle(video_fd[idx], (mouth_left_x, mouth_left_y), 1, (0, 255, 0), 4) 
            cv2.circle(video_fd[idx], (mouth_right_x, mouth_right_y), 1, (255, 0, 0), 4) # mouth right corner
#     break

In [ ]:
# Show the video with detected faces
idx = 0
for idx, frame in enumerate(video_fd):
#     cv2.imshow('face detection test', frame)
#     aa = cv2.waitKey(300)
    
#     if idx == 0:
#         a =input('press a key to start')
#     idx +=1
    title = 'Frame %d' % idx
    plt.figure()
    plt.title(title)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
# show the frame with no face detected
print("Number of frames with no face dectected : " + str(len(no_face)) )
for face in no_face:
    plt.figure()
    plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

In [ ]:
# show the detected faces
print("Number of faces dectected : " + str(len(crop_faces)) )
for face in crop_faces:
    plt.figure()
    plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

In [ ]:
# TO DO : save the results
# add metadata to the  detected faces : time stamps, confidence score, keypoints
# saves the hyperparameters margin ...